# Baseline - Most popular
User individual splitting of ratings

In [14]:
from bibrec.server.Utils import assign_popular_based_score
import pandas as pd
import numpy as np
import bibrec.server.evaluation as eval
from collections import defaultdict
import bibrec.server.Utils as Utils
import importlib
from sklearn.model_selection import train_test_split

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


In [15]:
ratings = ratings[:150000]

In [16]:
users_with_ratings = ratings["user_id"].unique()

train = pd.DataFrame(columns=ratings.columns)
test = pd.DataFrame(columns=ratings.columns)

# split data for each user individually
for user_id in users_with_ratings:
    mask = ratings["user_id"] == user_id
    user_ratings = ratings[mask]
    if len(user_ratings) < 3:
        continue
    train_user, test_user = train_test_split(user_ratings, test_size=0.25)
    train = pd.concat([train, train_user])
    test = pd.concat([test, test_user])



In [17]:
print("Total Ratings Count:", len(train) + len(test))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 129154
Train Data Ratings Count: 94371
Test Data Ratings Count: 34783


In [18]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['book_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'book_rating']]

In [19]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "book_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)

print(popular_books[:20])

              isbn13  vote_count  avg_rating  weighted_rating
13427  9780316666343         128    8.156250         8.152599
19825  9780385504201         104    8.461538         8.454145
6930   9780060928339          80    7.875000         7.872658
48088  9780971880108          76    4.302632         4.346562
31904  9780590353427          69    8.913043         8.895504
11729  9780312195519          66    8.696970         8.681870
2378    978044667227          63    8.174603         8.166957
36108  9780679764021          62    7.274194         7.280719
9974   9780142001745          61    8.639344         8.623956
27568  9780452282155          58    7.948276         7.943818
38612  9780743418171          57    8.052632         8.046298
13406  9780316601955          54    7.388889         7.394278
23532  9780440237228          53    7.716981         7.716394
23175  9780440211457          52    7.865385         7.861986
33521  9780671027360          51    8.215686         8.205486
13475  9

In [20]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['avg_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['book_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [21]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 7547


In [22]:
def def_value():
    return "Not Present"


top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [27]:
import time

importlib.reload(eval)
importlib.reload(Utils)
start_time = time.time()
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.0017676592844903438
--- Calculation time: 41.738121032714844 seconds ---


In [28]:
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.023394223911849556
--- Calculation time: 82.47995185852051 seconds ---
